In [36]:
#https://www.nalog.ru/opendata/7707329152-rsmp/


In [8]:
import glob
import xml.etree.cElementTree as et
import pymongo
from pymongo.write_concern import WriteConcern
import zipfile
import pandas as pd

In [9]:
def save_to_db(root):
    """
    Парсим файл xml. Добавляем в list каждый документ из файла, у которого 'КодРегион' равен '14'
    Все такие документы сохраняем в базу данных
    """
    i=0
    i_14=0
    okveddop_count = 0
    info = {}
    docs = []
    okveddop = []
    doc={}
    region_14 = False

    stage = 'Инфо'#о файле
    for elem in root.iter():
        
        #Документы
        if elem.tag == 'Документ':
            stage = 'Документ'
            if i >0:
                if okveddop: #есть дополнительные ОКВЭДы
                    doc['СвОКВЭДДоп'] = okveddop 

                # we save only data for region 14
                if region_14:
                    docs.append(doc)

            i = i+1
            doc={}
            okveddop=[]
            region_14 = False
            doc.update(info)

        if stage == 'Документ':
            if elem.tag == 'Документ':
                doc.update(elem.attrib)
                continue
            if elem.tag == 'СвОКВЭДДоп':
                okveddop.append(elem.attrib)
                continue
            if elem.tag == 'СведМН':
                if(elem.attrib['КодРегион']=='14'):
                    region_14 = True
                    i_14 = i_14+1
                
            doc[elem.tag] = elem.attrib

        #Информация о файле
        if stage == 'Инфо':
            info.update(elem.attrib)

    #save to database
    if docs:
        try:
            collection.insert_many(docs)
        except Exception as e:
            print("Error trying to insert collection:", type(e), e)

    return (i,i_14)# всего докуентов в файле, документов с 'КодРегион' равен '14'
    

In [10]:
# connect to the MongoDB
connection = pymongo.MongoClient("mongodb+srv://")

db = connection.rsmp     # attach to db
collection = db.docs.with_options(write_concern=WriteConcern(w=2))         # specify the collection

In [11]:
#открываем архивный файл
archivefile = zipfile.ZipFile('data-12102018-structure-08012016.zip')

list_14 = []
files = archivefile.namelist()
total_files = len(files)
print('Total: ', total_files)
curr_file = 1

#парсим каждый файл из архива
for name in files:
    xmlstr = archivefile.read(name)
    root = et.fromstring(xmlstr)
    i, i_14 = save_to_db(root)
    
    if i_14:
        list_14.append({'file_name': name, 'docs_in_file': i, 'docs_for_14':i_14})
    
    #to show progress
    if (curr_file % 100) == 0:
        print('File %d' % curr_file)
    
    curr_file = curr_file + 1
    
#закрываем файл
archivefile.close()

pd.DataFrame(list_14).to_excel('files_with_region_14.xls', index=False)# сохраняяем справочную информацию 

Total:  6714
File 100
File 200
File 300
File 400
File 500
File 600
File 700
File 800
File 900
File 1000
File 1100
File 1200
File 1300
File 1400
File 1500
File 1600
File 1700
File 1800
File 1900
File 2000
File 2100
File 2200
File 2300
File 2400
File 2500
File 2600
File 2700
File 2800
File 2900
File 3000
File 3100
File 3200
File 3300
File 3400
File 3500
File 3600
File 3700
File 3800
File 3900
File 4000
File 4100
File 4200
File 4300
File 4400
File 4500
File 4600
File 4700
File 4800
File 4900
File 5000
File 5100
File 5200
File 5300
File 5400
File 5500
File 5600
File 5700
File 5800
File 5900
File 6000
File 6100
File 6200
File 6300
File 6400
File 6500
File 6600
File 6700
